In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

epochs = 60
batch_size = 256
learning_rate = 1e-3

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:


transform = transforms.Compose([
    ### flip, rotate, crop
    # numpy to tensor
     transforms.ToTensor(),
    # 0~255 --> -1 ~ 1
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

trainset = torchvision.datasets.CIFAR10(root ='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root ='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True, num_workers=12)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.name = "SimpleCNN"
        self.conv_layer = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3)),
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3)),            
            nn.ReLU(inplace=True),
#             nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2), # subsampling
            
#             nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
#             nn.Dropout(p = 0.5),
            # 32*5*5 --> 64
            nn.Linear(32*5*5, 64),
            nn.ReLU(inplace=True),

            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            
            nn.Linear(32, 10)
        )
     

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(-1, 32*5*5)
        x = self.fc_layer(x)
        return x

import torchsummary
model = SimpleCNN()
torchsummary.summary(model, (3, 32, 32), device = 'cpu')
model = model.to(device)


In [3]:
class AlexNet(nn.Module) :
    def __init__(self) :
        super(AlexNet, self).__init__()
        self.name = "AlexNet"
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=(4, 4)),
            nn.ReLU(inplace=True),
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=(5, 5), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.fc_layer1 = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10)
        )

    def forward(self, x) :
        output = self.conv_layer1(x)
        output = self.conv_layer2(output)
        output = self.conv_layer3(output)
        output = output.view(-1, 9216)
        output = self.fc_layer1(output)
        return output
    
import torchsummary
model = AlexNet()
torchsummary.summary(model, (3, 32, 32), device = 'cpu')
model = model.to(device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 29, 29]           4,704
              ReLU-2           [-1, 96, 29, 29]               0
            Conv2d-3          [-1, 256, 29, 29]         614,656
              ReLU-4          [-1, 256, 29, 29]               0
         MaxPool2d-5          [-1, 256, 14, 14]               0
            Conv2d-6          [-1, 384, 14, 14]         885,120
              ReLU-7          [-1, 384, 14, 14]               0
            Conv2d-8          [-1, 384, 14, 14]       1,327,488
              ReLU-9          [-1, 384, 14, 14]               0
           Conv2d-10          [-1, 256, 14, 14]         884,992
             ReLU-11          [-1, 256, 14, 14]               0
        MaxPool2d-12            [-1, 256, 6, 6]               0
          Dropout-13                 [-1, 9216]               0
           Linear-14                 [-

In [4]:
def test_accuracy(model, test_dataloader, batch_size):
  
    with torch.no_grad():
        model.eval()
        total_correct =0
        total_cnt = 0

        for it_batch, data in enumerate(test_dataloader) :
            imges, labels = data

            imges = imges.to(device)
            # gt --> 확률벡터로 변환
            labels = torch.eye(10)[labels]  
            labels = labels.to(device)

            preds = model(imges)

            t1 = torch.argmax(labels, dim=1)
            t2 = torch.argmax(preds, dim=1)

            total_correct += (t1==t2).sum()
            total_cnt +=1

    # prt('correct [%d] acc [%.2f] ', total_correct, total_correct/(total_cnt*batch_size))

    return total_correct, total_cnt*batch_size


In [5]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for it_epochs in range(epochs):
    train_loss = 0
    loss_cnt = 0
    for it_batch, data in enumerate(train_dataloader) :

        model.train()
        imges, labels = data

        imges = imges.to(device)
        labels = torch.eye(10)[labels]  # 확률벡터로 변환
        labels = labels.to(device)

        preds = model(imges)

        optimizer.zero_grad()

        loss = criterion(preds, labels)
        train_loss += loss.item()
        loss_cnt += 1

        loss.backward()
        optimizer.step()

    prt = 'epoch %d train loss: %.2f' % (it_epochs, train_loss/loss_cnt)
    print(prt)
    total_correct, total_cnt = test_accuracy(model, test_dataloader, batch_size)
    prt = 'correct [%d] ##### acc [%.2f] ' % (total_correct, total_correct/total_cnt)
    print(prt)


epoch 0 train loss: 1.76
correct [5077] ##### acc [0.50] 
epoch 1 train loss: 1.24
correct [6321] ##### acc [0.62] 
epoch 2 train loss: 1.01
correct [6874] ##### acc [0.67] 
epoch 3 train loss: 0.86
correct [7111] ##### acc [0.69] 
epoch 4 train loss: 0.76
correct [7430] ##### acc [0.73] 
epoch 5 train loss: 0.68
correct [7675] ##### acc [0.75] 
epoch 6 train loss: 0.62
correct [7726] ##### acc [0.75] 
epoch 7 train loss: 0.57
correct [7829] ##### acc [0.76] 
epoch 8 train loss: 0.51
correct [7902] ##### acc [0.77] 
epoch 9 train loss: 0.48
correct [7961] ##### acc [0.78] 


KeyboardInterrupt: ignored

In [ ]:
def draw_model_result(img_sample, label_sample, pred_sample):
    img_sample = img_sample.numpy().transpose([1, 2, 0])
    img_sample = img_sample / 2 + 0.5 # -1~1 --> 0~1
    title_ = 'gt[%s], pred[%s]' % (classes[torch.argmax(label_sample)], classes[torch.argmax(pred_sample)])
    plt.title(title_)
    plt.imshow(img_sample)
    plt.show()

In [ ]:
with torch.no_grad():
    model.eval()

    data = next(iter(test_dataloader))
    imges, labels = data

    imges = imges.to(device)
    # gt --> 확률벡터로 변환
    labels = torch.eye(10)[labels]  
    labels = labels.to(device)

    preds = model(imges)

imges = imges.cpu()
labels = labels.cpu()
preds = preds.cpu()



for it_sample, img in enumerate(imges):
    draw_model_result(img, labels[it_sample], preds[it_sample])